In [1]:
import re
import pandas as pd

In [5]:
Review = "review_text"

In [3]:
df = pd.read_csv("../../kion_films_reviews.csv")
df.head()

,film_id,film_title,review_id,author_id,author_name,review_title,review_type,pos,neg,review_text,date
0,477647,Гостья,1755958,1414072,Lost__Soul,Прекрасное создание,NEGATIVE,241,123,\nНебезызвестный феномен современного кинемато...,2013-03-28 00:04:00
1,477647,Гостья,1755392,17062,rain 13,Идеальный мир паразитов,POSITIVE,138,62,"\nЕсли живы воспоминания, жив и ты (с)\n\nТема...",2013-03-27 11:21:00
2,477647,Гостья,1757153,423716,Aeger_Faber,Странница,NEGATIVE,153,90,"\nЕсли вы читаете эту рецензию, то, вероятно, ...",2013-03-29 16:59:00
3,477647,Гостья,1756803,1749081,клементина кручински,Копеечное счастье Стефани Майер.,NEGATIVE,141,84,"\nВсем, кто купился на мерцающий электрический...",2013-03-28 23:54:00
4,477647,Гостья,1761571,1685,korsar45,Безаналитическая история Говарда Лавью,NEUTRAL,46,12,\nШёл 2017-й год. Земля стремительно очищалась...,2013-04-04 19:33:00


In [11]:
len(df[df[Review].str.startswith("\n")]) == len(df)

True

In [6]:
df[df[Review].str.contains("\u2028")] # line separator — разделитель строк

,film_id,film_title,review_id,author_id,author_name,review_title,review_type,pos,neg,review_text,date
8319,86208,«V» значит Вендетта,1469652,1998839,Юлия Михайлова,"Под этой маской нечто большее, чем плоть. Под ...",POSITIVE,16,2,"\nФильм, который заставляет задуматься. Фильм,...",2012-03-10 00:44:00
31363,397496,Бросок кобры,577559,1,white kvarz,Нам всем есть о чем сожалеть (с),NEUTRAL,41,24,\nВот и подоспел новый летний блокбастер 'Брос...,2009-08-11 12:16:00
59088,424994,Лига справедливости,2618505,14568620,ComradeV,Лига НеСправедливости.,NEUTRAL,1,1,\nСо дня мировой премьеры Лиги я уворачивался ...,2017-11-17 15:25:00
72264,522036,"Высоцкий. Спасибо, что живой",1417867,1778474,Kapitan KUK,Высоцкий. Простите за цинизм,NEGATIVE,23,13,\n«Я не люблю холодного цинизма...»\n\nФильм п...,2012-01-13 03:41:00
117116,839,Спирит: Душа прерий,1198958,818240,amor-93,Here i am...,POSITIVE,31,5,"\nЕсть мультики, которые западают в душу и ест...",2011-04-27 23:00:00
294617,1151373,В метре друг от друга,2795077,22191884,sasha.bobykina,"Я всё время жила чтобы лечится, а не лечилась,...",POSITIVE,7,14,\nТрейлер «в метре друг от друга» сразу же рис...,2019-05-04 18:18:00
319350,673910,Враг,2647129,14640529,Huker1,О чем этот фильм? — О моем подсознании,POSITIVE,10,1,\nКак и любой фильм одного из лучших современн...,2018-02-03 20:44:00
350231,718222,Звёздные войны: Скайуокер. Восход,2908440,5341882,EugeneHarris,Охота на ведьм #14,NEGATIVE,17,1,"\nЯ, что называется, в шоке. Красный цвет и «З...",2020-05-05 16:55:00


In [7]:
t = pd.DataFrame(data={
    "text": [
        "наблюдать три часа за героями. \r\n\r\nНе судите, да не",
        "Каждый из них прошёл свою зелёную милю\r\n\r\nПомимо превосходной игры",
        "Amogus!!!   \r\n\r\nAmogus...",
        "Переосмысление…\r\n<i>Начало</i> легенды. <i>ого</i>",
        "It`s a new world\u2028It`s a new start\u2028It`s",
        "«Дуэль».\t. \t. Дальнейшее – мнение."
    ]
})

def func(match: re.Match):
    group = match.group(1)
    if group == "":
        return ". "
    return f"{group[0]} "

def tags(match: re.Match):
    return match.group(1)

t["text"] = t["text"].str.replace("([…:!\.\?]?) *(?:\r\n)+", func, regex=True)
t["text"] = t["text"].str.replace("<\w+>([^<>]+)<\/\w+>", tags, regex=True)
t["text"] = t["text"].str.replace(" *\t *", " ", regex=True)
t["text"] = t["text"].str.replace("\u2028", ", ")
t

,text
0,"наблюдать три часа за героями. Не судите, да не"
1,Каждый из них прошёл свою зелёную милю. Помимо...
2,Amogus!!! Amogus...
3,Переосмысление… Начало легенды. ого
4,"It`s a new world, It`s a new start, It`s"
5,«Дуэль». . . Дальнейшее – мнение.


In [ ]:
def replace_repeated_html_tags(series: pd.Series):
    new = series
    while True:
        old = new
        new = old.str.replace("<\w+>([^<>]*)<\/\w+>", tags, regex=True)
        if (old == new).all():
            break
    return new

In [12]:
filtered = df.copy(deep=True)
url_regex = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
filtered[Review] = filtered[Review].str.lstrip("\n")
filtered[Review] = filtered[Review].str.replace(url_regex, "", regex=True)
filtered[Review] = filtered[Review].str.replace("([…:!\.\?]?) *(?:[\r\n])+", func, regex=True)
filtered[Review] = replace_repeated_html_tags(filtered[Review])
filtered[Review] = filtered[Review].str.replace("<\/?[\w\d\. =':\/]+>", "", regex=True)
filtered[Review] = filtered[Review].str.replace("[\u0301\u200b\u2122\ufeff]", "", regex=True)
filtered[Review] = filtered[Review].str.replace(" *\t *", " ", regex=True)
filtered[Review] = filtered[Review].str.replace("\u2028", ", ")
filtered[Review] = filtered[Review].str.replace("\u00E9", "e")
filtered.to_csv("top_250_films_reviews_filtered.csv", index=False)
filtered

: 

: 

In [11]:
tm = "\u2122" # ™
spzw = "\u200b" # space
nbsp = "\ufeff" # space
nsign = "\u2116" # №
latin = "\u0020-\u007E" # a-zA-Z
punct = "\u00A0-\u00BE\u2010-\u2049"
cyril = "\u0400-\u04FF" # а-яА-я
accent = "\u0301" # ударение
g = filtered[filtered["description"].str.contains(f"[^{latin}{punct}{cyril}{spzw}{nsign}{tm}{accent}{nbsp}\u00E9]", regex=True)]
print(len(g))
g.head()

3


,film_id,review_id,type,date,pos,neg,author,title,description
728,370,376971,POSITIVE,2009-02-07T17:35:00,84,10,Vanilla_hypocrite,Глубокий и красивый фильм «Унесенные призраками»,"Унесенные призраками (яп. 千と千尋&- #12398;神隠し, С..."
3707,9691,588095,POSITIVE,2009-08-19T22:28:00,122,83,Zmey-Aspid,Это утро для штандартенфюрера Штирлица началос...,У него на столе неожиданно появилось несколько...
9005,596125,1901554,POSITIVE,2013-10-29T08:53:42,34,4,Live to tell,Как обрести соперника и заставить всех тобой в...,"История, которая легла в основу сценария «Гонк..."


In [102]:
text = df.iloc[55]["description"]
test = pd.DataFrame(data={"text": [text]})
test.iloc[0]["text"]

'Посмотреть этот фильм, это то же самое как прожить жизнь в том времени и месте, где происходит действие. Ты представляешь, как бежишь по гнилой и когда-то зеленой и красивой земле, пропитанной кровью не в чем не виноватых людей. Ты будто сам вдыхаешь этот мертвый воздух, который не дает тебе спокойно дышать. <I><B>Тебя словно хватают за сердце и сдавливают со всей силы в моменты, когда проливается невинная и почти тебе родная кровь. </B></I>\r\n\r\n<I><B>Это страшно</B></I>. Видеть то, как люди, которые тоже имеют сердце, которые тоже счастливы в этом мире и хотят светлого и ясного будущего, теряют все это практически не из за чего. Люди верят, надеются, пытаются менять ход событий и сберечь себя и тех, ради кого они живут. Ради тех, кто вселяет надежду и существование в них самих. Это катастрофа. Это полное безрассудство. Это кошмар, который будит происходить до тех пор, пока светит солнце.\r\n\r\n<I><B> Добро равноправно злу. На все зло есть добро, а на добро – есть зло.</B></I> В н

In [103]:
test["text"] = test["text"].str.replace("([…:!\.\?]?) *(?:\r\n)+", func, regex=True)
test["text"] = replace_repeated_html_tags(test["text"])
test.iloc[0]["text"]

'Посмотреть этот фильм, это то же самое как прожить жизнь в том времени и месте, где происходит действие. Ты представляешь, как бежишь по гнилой и когда-то зеленой и красивой земле, пропитанной кровью не в чем не виноватых людей. Ты будто сам вдыхаешь этот мертвый воздух, который не дает тебе спокойно дышать. Тебя словно хватают за сердце и сдавливают со всей силы в моменты, когда проливается невинная и почти тебе родная кровь. . Это страшно. Видеть то, как люди, которые тоже имеют сердце, которые тоже счастливы в этом мире и хотят светлого и ясного будущего, теряют все это практически не из за чего. Люди верят, надеются, пытаются менять ход событий и сберечь себя и тех, ради кого они живут. Ради тех, кто вселяет надежду и существование в них самих. Это катастрофа. Это полное безрассудство. Это кошмар, который будит происходить до тех пор, пока светит солнце.  Добро равноправно злу. На все зло есть добро, а на добро – есть зло. В нашей жизни, почему-то большинство замечают и откладываю

In [104]:
text = df.iloc[2650]["description"]
test = pd.DataFrame(data={"text": [text]})
test.iloc[0]["text"]

"Данный фильм - не более чем калька, снятая с малоизвестного неискушенному зрителю Японского кинофильма. Впрочем, вот на него ссылка по кинопоиску (кто захочет, тот найдет и посмотрит): \r\n\r\n<a href='http://www.kinopoisk.ru/level/1/film/38145/' class='all'>http://www.kinopoisk.ru/level/1/film/38145/</a>\r\n\r\nУ меня лично данное кино было еще на видеокассете (фильм снят в 1987 году).\r\n\r\nФильм более чем великолепный, и гораздо круче, чем американская подделка! Интересно, а что было бы, если бы американская киноиндустрия сняла  дубль нашей отечественной кинохи 'Белый Бим, черное ухо' (уверен, что многие об этом фильме и понятия не имеют)? \r\n\r\nСоветую НАЙТИ и посмотреть изначальную версию... Это того стоит..."

In [105]:
test["text"].str.replace("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", regex=True).values

array(["Данный фильм - не более чем калька, снятая с малоизвестного неискушенному зрителю Японского кинофильма. Впрочем, вот на него ссылка по кинопоиску (кто захочет, тот найдет и посмотрит): \r\n\r\n<a href=' class='all'>\r\n\r\nУ меня лично данное кино было еще на видеокассете (фильм снят в 1987 году).\r\n\r\nФильм более чем великолепный, и гораздо круче, чем американская подделка! Интересно, а что было бы, если бы американская киноиндустрия сняла  дубль нашей отечественной кинохи 'Белый Бим, черное ухо' (уверен, что многие об этом фильме и понятия не имеют)? \r\n\r\nСоветую НАЙТИ и посмотреть изначальную версию... Это того стоит..."],
      dtype=object)

In [67]:
filtered.iloc[367]["description"]

"<I><I>Знаете, чем отличается советский кинематограф от современного российского? Тем, что 30-40 лет назад создавались шедевры на все времена, комедии, которые никогда не надоест смотреть. 36 лет назад вышел один из таких фильмов - комедия Леонида Гайдая 'Иван Васильевич меняет профессию'. В чём же секрет успеха? Всем понятные и непошлые шутки, яркие, запоминающиеся персонажи (Бунша и Милославский выше всех похвал), фразы, ставшие народными афоризмами: - Танцуют все! - Всё, что нажито непосильным трудом - всё погибло. - Положь трубку! Задавлю, шляпа! - Житие мое... - Я требую продолжения банкета. - Вы с магнитофоном пришли к Шпаку? - Тьфу на вас. - Граждане! Храните деньги в сберегательной кассе. Если, конечно, они у вас есть... - И тебя вылечат, и тебя вылечат, и меня вылечат. А песни с незаурядным текстом давно 'прописались' в каждом сборнике караоке. Но самый главный плюс - фильм можно смотреть и взрослым, и детям. И можно опустить всякие - разные неточности, типа 'Тысяча пятьсот тр

In [107]:
filtered.iloc[9005]["description"]

'История, которая легла в основу сценария «Гонки», - это известные всем болельщикам Формулы-1 события, но, как справедливо отмечали первые же партии зрителей, смотреть можно и абсолютно незамутненными глазами, т.е. будучи ни в зуб ногой про Большой цирк. Зато можно сразу понять, почему Формулу-1 так называют: в наши дни, когда все напичкано электроникой, для пилотов риска меньше, а гонорары выше, но в 1970-х гг., когда гонялись Лауда и Хант, зрители за свои деньги могли рассчитывать на заезды не на жизнь, а на смерть. Нельсон Пике, трехкратный, кстати, чемпион мира и один из четверых  коллег по цеху, кого в своей автобиографии Ники Лауда называет другом, говорил, что отдает себе отчет, что  погибнет  на трассе, и его мать это осознает. К счастью, для Пике все обошлось, но такой вот краткий экскурс в историю позволяет составить себе впечатление о том, насколько беспощадной была когда-то Формула-1. В фильме Рона Ховарда четко показано, что трус  не садится в болид.\n\nОснована-то «Гонка»

In [12]:
tt = filtered[filtered["description"].str.contains(f"[\u00E9]", regex=True)]
tt

,film_id,review_id,type,date,pos,neg,author,title,description


In [13]:
g.iloc[0]["description"]

'Унесенные призраками (яп. 千と千尋&- #12398;神隠し, Сэн то Тихиро но камикакуси) это анимэ с глубоким смыслом и захватывающим сюжетом. Этот фильм зашел мне куда-то глубоко в душу. Даже в просмотре многих фильмов я так не переживаю, как переживала за судьбу героев этого мультфильма. В 2003 году «Унесенные призраками» получили «Оскар» в категории «Лучший анимационный полнометражный фильм», опередив «Ледниковый период», «Лило и Стич», «Спирита» и «Планету сокровищ». Этот мультфильм завоевал приз «Золотой медведь» Берлинского кинофестиваля в 2002 году, стал лучшим фильмом года по мнению Японской и Гонконгской кино академий и был назван лучшей анимационной лентой года Национальным советом критиков Америки. В 2003 году картина была удостоена четырёх американских премий «Энни» в области анимационного кино в номинациях: «Лучший фильм», «Лучшая режиссура», «Лучшая музыка», «Лучший сценарий». Мир, показанный в этом анимэ, похож на Токийский музей, чем на современность. У режиссера Миядзаки всегда вызы

In [14]:
"é".encode()

b'\xc3\xa9'

In [15]:
df.iloc[9498]["description"]

'Переосмысление классического комикса. <i>Начало</i> легенды. Знаменитый фильм моего любимого режиссера Кристофера Нолана «Бэтмен: Начало». Истоки легенды Брюса Уэйна – знаменитого миллиадера, наследника огромной корпорации и в то же время мстителя в маске, карающего преступников под покровом ночи. \r\n\r\n<i>Реалистичность.</i>\r\n\r\nФильм не имеет ничего общего с ранними экранизациями, которые выглядели как разноцветный фантик. Нолан показал Готем как обыкновенный американский мегаполис со своими правилами, законами, и…преступностью. Боссы мафии, наркодилеры, преступники, держащие весь город в страхе. И на фоне этого мрака берет свое начало великая история про борьбу одного человека не только с преступниками Готема, но и самим с собой. Его зовут Брюс Уэйн. Отсидев в тюрьме практически на крае света, он научился выдержке и терпению. Мудрый наставник помог постичь боевые искусства в совершенстве. Испытания закалили его характер и придали силу воли.\r\n\r\n<i>Мрачность. </i>\r\n\r\nГря

In [16]:
filtered.iloc[9498]["description"]

'Переосмысление классического комикса. Начало легенды. Знаменитый фильм моего любимого режиссера Кристофера Нолана «Бэтмен: Начало». Истоки легенды Брюса Уэйна – знаменитого миллиадера, наследника огромной корпорации и в то же время мстителя в маске, карающего преступников под покровом ночи. Реалистичность.. Фильм не имеет ничего общего с ранними экранизациями, которые выглядели как разноцветный фантик. Нолан показал Готем как обыкновенный американский мегаполис со своими правилами, законами, и…преступностью. Боссы мафии, наркодилеры, преступники, держащие весь город в страхе. И на фоне этого мрака берет свое начало великая история про борьбу одного человека не только с преступниками Готема, но и самим с собой. Его зовут Брюс Уэйн. Отсидев в тюрьме практически на крае света, он научился выдержке и терпению. Мудрый наставник помог постичь боевые искусства в совершенстве. Испытания закалили его характер и придали силу воли. Мрачность. . Грязные улицы, на которых творится беззаконие, разв

In [17]:
filtered.iloc[7678 - 2]["description"]

"<i>Что ещё за Канзас Сити Шаффл? Канзас Сити Шаффл — это когда все смотрят направо, а ты идёшь налево.</i>. <i>Было время:</i> один хороший парень, желая, чтобы его сын поступил в Гарвард на медицинский, а любимая жена обзавелась домиком с лужайкой, пошёл к левому букмекеру и поставил 20 тысяч долларов на 'верную' лошадь. Он считал, что риск оправдан. Но забыл, что ставка его была условная. И тем самым он задолжал людям, у которых в долг лучше вообще не брать. Накаченная химией лошадь до финиша не дошла. А хорошего парня нашли люди очень нехорошие, которые убили всех, кто имел отношение к этой истории: лошадь, букмекера, хорошему парню надели пакет на голову, а его жену застрелили в упор из дробовика. <i>Твою мать! Чёрт! Господи! Всё верно, именно в таком порядке.</i>. Наши дни: просто парень Слевин Келевра (Джош Хартнетт) теряет работу, жильё, девушку. Он приезжает в Нью-Йорк к другу Нику Фишеру. По приезде ему ломают нос и отнимают бумажник с деньгами и документами. Ника в дома не о

In [18]:
filtered.iloc[7678 - 2][["description"]].str.replace("<\w+>([^<>]+)<\/\w+>", tags, regex=True)["description"]

"Что ещё за Канзас Сити Шаффл? Канзас Сити Шаффл — это когда все смотрят направо, а ты идёшь налево.. Было время: один хороший парень, желая, чтобы его сын поступил в Гарвард на медицинский, а любимая жена обзавелась домиком с лужайкой, пошёл к левому букмекеру и поставил 20 тысяч долларов на 'верную' лошадь. Он считал, что риск оправдан. Но забыл, что ставка его была условная. И тем самым он задолжал людям, у которых в долг лучше вообще не брать. Накаченная химией лошадь до финиша не дошла. А хорошего парня нашли люди очень нехорошие, которые убили всех, кто имел отношение к этой истории: лошадь, букмекера, хорошему парню надели пакет на голову, а его жену застрелили в упор из дробовика. Твою мать! Чёрт! Господи! Всё верно, именно в таком порядке.. Наши дни: просто парень Слевин Келевра (Джош Хартнетт) теряет работу, жильё, девушку. Он приезжает в Нью-Йорк к другу Нику Фишеру. По приезде ему ломают нос и отнимают бумажник с деньгами и документами. Ника в дома не оказывается. Зато Слев